### Устанавливаем и запускаем Vertica

In [ ]:
pip install vertica-python

In [3]:
import os

In [4]:
os.system('docker run -d -p 5433:5433 jbfavre/vertica:latest')

0

### Подключаемся к БД

In [1]:
import vertica_python

In [2]:
connection_info = {
    'host': 'localhost',
    'port': 5433,
    'user': 'dbadmin',
    'password': '',
    'database': 'docker',
    'autocommit': True,
} 

### Проводим Миграции

In [3]:
connection =  vertica_python.connect(**connection_info)
cursor = connection.cursor()

In [4]:

cursor.execute("""CREATE TABLE movie_view (
    id IDENTITY, user_uuid VARCHAR(256), movie_uuid VARCHAR(256), movie_progress VARCHAR(256), movie_length VARCHAR(256), event_time VARCHAR(256));
""")

DuplicateObject: Severity: ROLLBACK, Message: Object "movie_view" already exists, Sqlstate: 42710, Routine: outputAlreadyExistsEreport, File: /data/qb_workspaces/jenkins2/ReleaseBuilds/Grader/REL-9_2_0-x_grader/build/vertica/Commands/DDL.cpp, Line: 22038, Error Code: 4213, SQL: 'CREATE TABLE movie_view (     id IDENTITY, user_uuid VARCHAR(256), movie_uuid VARCHAR(256), movie_progress VARCHAR(256), movie_length VARCHAR(256), event_time VARCHAR(256)); '

### Заполняем БД на 1 000 000 записей

In [9]:
import time
import pandas
from pydantic import parse_obj_as, BaseModel
import datetime
from typing import List
import random
import uuid

In [10]:
movie_uuids = [str(uuid.uuid4()) for _ in range(1000)]
user_uuids = [str(uuid.uuid4()) for _ in range(2000)]

In [13]:
length = random.randint(600, 90000)
movies_gen = ((
    random.choice(user_uuids), random.choice(movie_uuids), str(random.randint(0, length)), str(length), str(datetime.datetime.now())) for _ in range(10_000_000)
)


In [ ]:
%%time
step = 10000
for _ in range(100):
    cursor.executemany("""INSERT INTO movie_view (
                       user_uuid, movie_uuid, movie_progress, movie_length, event_time)
                       VALUES (?, ?, ?, ?, ?)""",
                       [next(movies_gen) for _ in range(step)],
                       use_prepared_statements=True) 


In [5]:
connection.rollback()
cursor.execute('SELECT COUNT(*) from movie_view')

In [6]:
cursor.fetchall()

[[60000]]

За час на одном потоке было загружено 60000 записей